In [1]:
import torch
import math
import numpy as np
import json
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd

In [2]:
import sys
sys.path.append('..')
from ridesharing_simulation import *

In [3]:
start_week = 0
T0_pricing_params = (5, 0.78, 1.82)
lr = 0.01

In [4]:
#simulation_week1 = WeeklySimulation(start_week, lr, T0_pricing_params)

In [6]:
total_weeks = 1

for w in tqdm(range(total_weeks)):
    simulation_this_week = WeeklySimulation(w, lr, T0_pricing_params)
    for d in range(7):
        drivers_this_week_today = simulation_this_week.simulate_supply()
        requests_this_week_today = simulation_this_week.simulate_demand()

        simulation_this_week.request_driver_matching(verbose=1)
        simulation_this_week.update_gamma_distns()

100%|██████████| 1/1 [00:00<00:00,  3.04it/s]


In [8]:
demand_columns = ["request_timestamp", "x_start", "y_start", "x_end", "y_end"
                    , "rider_idx", "trip_start_subblock_id", "trip_end_subblock_id"]

In [9]:
supply_columns = ["idle_start_timestamp", "idle_duration", "idle_x_start", "idle_y_start"
                   , "driver_idx", "idle_start_subblock_id"]

In [11]:
total_weeks = 10

stdout_logger = utils.create_logger_custom("weekly_daily_main_function")

for w in tqdm(range(total_weeks)):
    simulation_this_week = WeeklySimulation(w, lr, T0_pricing_params)
    for d in range(7):
        drivers_this_week_today = simulation_this_week.simulate_supply()
        requests_this_week_today = simulation_this_week.simulate_demand()

        lst1 = []
        for row in simulation_this_week.D_Requests:
            row_dict = {demand_columns[i]: row[i].item() for i in range(row.size(0))}
            lst1.append(row_dict)

        # Save each dictionary as a JSON object in a JSON file
        with open('../data/10_weeks/demand_week_'+str(w)+'.json', 'w') as json_file:
            json.dump(lst1, json_file, indent=4)

        lst2 = []
        for row in simulation_this_week.S_Drivers:
            row_dict = {supply_columns[i]: row[i].item() for i in range(row.size(0))}
            lst2.append(row_dict)

        # Save each dictionary as a JSON object in a JSON file
        with open('../data/10_weeks/supply_week_'+str(w)+'.json', 'w') as json_file:
            json.dump(lst2, json_file, indent=4)

        simulation_this_week.request_driver_matching(verbose=1)
        simulation_this_week.update_gamma_distns()
        std_out_entry = {
            "week": w,
            "day": d,
            "drivers_size": list(drivers_this_week_today.size()),
            "requests_size": list(requests_this_week_today.size())
        }
        stdout_logger.debug(json.dumps(std_out_entry))

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:04<00:00,  2.07it/s]


In [13]:
for i in range(10):
    trips_log_df = pd.read_json("../data/10_weeks/logfile_matched_trips_week_"+str(i)+".log", lines=True)
    demand_log_df = pd.read_json("../data/10_weeks/demand_week_"+str(i)+".json")
    supply_log_df = pd.read_json("../data/10_weeks/supply_week_"+str(i)+".json")
    trips_log_df.to_csv("../data/10_weeks/matched_trips_week_"+str(i)+".csv", index=False)
    demand_log_df.to_csv("../data/10_weeks/demand_week_"+str(i)+".csv", index=False)
    supply_log_df.to_csv("../data/10_weeks/supply_week_"+str(i)+".csv", index=False)


### It takes ~0.3sec per week, so ~3000(0.8hr) secs for 10K weeks, but as we have more rounds, the multiple increases
### remember to delete previous log files before rerunning a new simulation

In [5]:
total_weeks = 100

for w in tqdm(range(total_weeks)):
    simulation_this_week = WeeklySimulation(w, lr, T0_pricing_params)
    for d in range(7):
        drivers_this_week_today = simulation_this_week.simulate_supply()
        requests_this_week_today = simulation_this_week.simulate_demand()
        simulation_this_week.request_driver_matching()
        simulation_this_week.update_gamma_distns()

100%|██████████| 100/100 [00:29<00:00,  3.36it/s]


In [6]:
total_weeks = 1000

for w in tqdm(range(total_weeks)):
    simulation_this_week = WeeklySimulation(w, lr, T0_pricing_params)
    for d in range(7):
        drivers_this_week_today = simulation_this_week.simulate_supply()
        requests_this_week_today = simulation_this_week.simulate_demand()
        simulation_this_week.request_driver_matching()
        simulation_this_week.update_gamma_distns()

100%|██████████| 1000/1000 [06:34<00:00,  2.53it/s]
